In [ ]:
from openfermion import QubitOperator
import numpy
import sys
sys.path.append('source')
from simulator import (
    pylove_simulation,
    tr,
    two_qubit_depolarizing,
    single_qubit_depolarizing,
    postselect)

# Trivial example

### The graph state
Before working on something serious, let us learn to use the simulator on a trivial example. In that spirit, let us consider a graph state on the following geometry.

![graphstate](graphstate.png)

According to [wikipedia](https://en.wikipedia.org/wiki/Graph_state), it has the following stabilizer generators $\mathtt{g_x}$: 

In [ ]:
gx = {
    0: QubitOperator('X0 Z1 Z5'),
    1: QubitOperator('X1 Z0 Z2 Z4'),
    2: QubitOperator('X2 Z1 Z3'),
    3: QubitOperator('X3 Z2 Z4 Z8'),
    4: QubitOperator('X4 Z1 Z3 Z5 Z7'),
    5: QubitOperator('X5 Z0 Z4 Z6'),
    6: QubitOperator('X6 Z5 Z7'),
    7: QubitOperator('X7 Z4 Z6 Z8'),
    8: QubitOperator('X8 Z3 Z7'),
}

In $\mathtt{g_y}$, an alternative version of $\mathtt{g_x}$, all Pauli $X$ operators are turned into $Y$ operators. Elements of the two sets obey the commutation relations
$$
\mathtt{g_x}[j]\;\mathtt{g_y}[k] = (-1)^{\delta_{jk}} \;\mathtt{g_y}[k]\;\mathtt{g_x}[j]\, .
$$

In [ ]:
gy = {
    0: QubitOperator('Y0 Z1 Z5'),
    1: QubitOperator('Y1 Z0 Z2 Z4'),
    2: QubitOperator('Y2 Z1 Z3'),
    3: QubitOperator('Y3 Z2 Z4 Z8'),
    4: QubitOperator('Y4 Z1 Z3 Z5 Z7'),
    5: QubitOperator('Y5 Z0 Z4 Z6'),
    6: QubitOperator('Y6 Z5 Z7'),
    7: QubitOperator('Y7 Z4 Z6 Z8'),
    8: QubitOperator('Y8 Z3 Z7'),
}

Graph states are fully constrained by their stabilizers and so there is no computational subspace to play with. However, we can use the sets $\mathtt{g_x}$ and $\mathtt{g_y}$ to define a $[[9,5,1]]$ quantum code, where 
$$
 \mathtt{g_x}[k] \; \text{for} \;k=1,3,5,7  \tag{1}
$$
 is the set of stabilizer generators. All other Pauli stings in $\mathtt{g_x}$ and $\mathtt{g_y}$, except for $\mathtt{g_y}[1]$, $\,\mathtt{g_y}[3]$, $\,\mathtt{g_y}[5]$ and $\mathtt{g_y}[7]$,  generate the logical operators of the system. Let us write down the stabilizers for later use.

In [ ]:
stabs = [gx[x] for x in (1, 3, 5, 7)]
print(stabs)

Let us say we want to simulate the circuit

$$
\exp\!\left(i \; 0.144 \; \mathtt{g_y}[0]\cdot\mathtt{g_x}[2] \right) \quad \exp\!\left(i \; 0.256 \; \mathtt{g_x}[0]\cdot\mathtt{g_x}[6]\cdot \mathtt{g_y}[2]\cdot\mathtt{g_x}[8] \right) \quad \exp\!\left(-i \; 0.296 \; \mathtt{g_x}[4]\cdot\mathtt{g_y}[8] \right)\tag{2}
$$

on the system. We would input such a circuit into the simulator as a list containing `QubitOperator` objects of the form:

In [ ]:
circuit = [
    -.296 * gx[4] * gy[8],
    .256 * gx[0] * gx[6] * gy[2] * gy[8],
    .144 * gy[0] * gx[2]]

Of course, we need a state to start from -- it has to lie within the code space of the stabilizer set. An easy way to prepare such a state on the quantum computer is to measure the stabilizer generators. To constrain the logical state, a.k.a. the computational state, we measure a few operators more. A set of logical operators is given to be confined along with the stabilizer generators. Let us say we want to simulate the circuit $(2)$ on an initial state defined by the stabilizer generators $(1)$ plus the logical operators:
$$
\phantom{+}i \mathtt{g_y}[0]\cdot\mathtt{g_x}[0] = \phantom{+}Z_0 ,\\
-i \mathtt{g_y}[2]\cdot\mathtt{g_x}[2] = -Z_2 , \\
\phantom{+}i \mathtt{g_y}[4]\cdot\mathtt{g_x}[4] = \phantom{+}Z_4 ,\tag{3}\\
-i \mathtt{g_y}[6]\cdot\mathtt{g_x}[6] = -Z_6 ,\\
-i \mathtt{g_y}[8]\cdot\mathtt{g_x}[8] = -Z_8 .
$$

In this example, we can identify the qubits $0, 2, 4, 6, 8$ as the computational subspace, the qubits that can actually store information. For the particular choice of $(3)$, we initialize the computational space in the configuration $|01011\rangle$, where the value of each bit is controlled by the sign of the respective $Z$ operator in $(3)$.
We add the logical operators to another list in the input format of the simulator.

In [ ]:
log_ops = [
    QubitOperator('Z0'),
    QubitOperator('Z2', -1),
    QubitOperator('Z4'),
    QubitOperator('Z6', -1),
    QubitOperator('Z8', -1)]

### Simulation and evaluation: a dry run

With the routine `pylove_simulation` we can now reconstruct the density matrix by running this circuit 100 times without specifying any noise.

In [ ]:
graphstate = pylove_simulation(
    stabilizers=stabs,
    logical_operators=log_ops,
    quantum_circuit=circuit,
    shots=100
)

The resulting object `graphstate`, is an instance of the class `pylove_state`. It retains a dense matrix of the state's logical representation via the attribute `.state`. Note how the density matrix has the dimension $32 \times 32$, it is a representation of the five logical qubits, rather than all 8 physical qubits.

The stabilizer generators can be accessed with the attribute `.stabs`. These generators are not necessarily the ones we input, but they generate the same group. The generators in `graphstate.stabs` directly link the to numerical representation of `graphstate.state`.

In [ ]:
print(numpy.shape(graphstate.state))
print(graphstate.stabs)

The object also holds the state of the simulation without noise.
The method `.ideal()` turns the ideal state into an instance of `pylove_state`. 

We can now use the simulator's trace function `tr` to calculate the fidelity of ideal result and the simulation.
The trace function always takes two arguments, where one of them can be any `pylove_state` and the other should either 
- an **ideal** state, if we want to calculate the fidelity (the trace of two mixed states is not their fidelity), or
- a `QubitOperator`, if we want to compute an expectation value.

In [ ]:
tr(graphstate.ideal(), graphstate)

The fidelity appears to be 1 (to a very high precision). This is because we have not specified any noise, and so we have run the circuit 100 times without noise. The `graphstate` and `graphstate.ideal()` are identical.

Before we run the simulation with some noise, let us look at some expectation values of a stabilizer, a logical operator and an error operator with respect to this state. 

In [ ]:
print('<stabilizer> = ',  tr(graphstate, stabs[1]))
print('<logical_op> = ',  tr(graphstate, QubitOperator('Z0')))
print('<error> = ',  tr(graphstate,  gy[1]))


The expectation values of all stabilizers are one. We are in the code space.
The expectation value of the logical operator $Z_0$ was $+1$ after the state preparation, but has changed throughout the circuit.
The expectation value of an error operator (an operator that would anticommute with stabilizers) is always zero, as it has lead us outside the codespace.



### Simulation with noise

Let us now give the simulation some noise. The simulator deploys noise statistically within the circuit and during the state preparation. We distinguish two types of Pauli noise;
- wire noise, a single qubit noise happening to all quantum wires in the all the time and
- gate noise, a two-qubit noise affecting CNot gates.

Depolarizing and dephasing noise are already implemented, but we will talk about noise customization somewhat later. For now, let us run depolarizing noise for wires and gates at a rate of $0.05$. We also increase the number of shots.

In [ ]:
new_graphstate = pylove_simulation(
    stabilizers=stabs,
    logical_operators=log_ops,
    quantum_circuit=circuit,
    shots=500,
    wire_noise=single_qubit_depolarizing(rate=.005),
    gate_noise=two_qubit_depolarizing(rate=.005),
    mode='all'
)

Note that we have set some keyword parameter `mode` to `'all'`. This keyword basically sets the attribute `mode` of the `pylove_state` resulting from the simulation. In a way, `mode` determines which kind of postselection is run after the algorithm and in which format the density matrix is stored.
 `pylove_simulation` allows for 3 possible settings of `mode`:

- **`mode = code`** : This is the default setting. The results are postselected on the code space, so only the codespace density matrix is saved, and it is normalized to trace 1. Shots in which result in a different syndrome pattern are discarded. The  `.state` attribute of a `pylove_state` object based on a $[[n,k,d]]$ code is a $2^k \times 2^k$ `numpy.ndarray`.
- **`mode = all`** : This is the most general setting. All the shots are saved. The  `.state` attribute of a `pylove_state` object based on a $[[n,k,d]]$ code is a  `numpy.ndarray` of dimension
$$
\underbrace{2 \times 2 \times 2 \times ... \times 2}_{(n-k)\; \text{times}} \times 2^k \times 2^k .
$$ 
- **`mode = custom`** : Custom setting. Requires to pass a sequence of syndrome patterns to the keyword parameter `block_numbers`. The required format for a single syndrome pattern of an $[[n,k,d]]$ code is a sequence of length $n-k$ with integer entries $0$ or $1$. The simulation will only keep the density matrix subblocks with the specified syndrome patterns, and the others will be discarded. Passing a sequence with all entries $0$ for instance will ensure that the code space density matrix is retained. The `.state` attribute of a `pylove_state` object is a dictionary whose keys are the syndromes passed to `block_numbers`, but reformulated so they correspond to the stabilizers from `.stabs` (data type `tuple`). The values of the table are $2^k \times 2^k$ `numpy.ndarray`s, the corresponding density matrix subblocks.

Note that a `pylove_state` instance `x` with `x.mode == 'all'` or `'custom'` is defined such that the density matrix subblock corresponding to the syndrome pattern `syndr` is accessible via `x[syndr]`, as long as `syndr` is of data type `tuple`.

Having selected the mode `'all'`, we are basically not doing any postselection. The entire state is retrieved within `new_graphstate`. The fidelity with the ideal state has decreased dramatically.

In [ ]:
tr(new_graphstate, new_graphstate.ideal())

Imagine a simple experiment with postselection. While measuring observables, we would also always measure stabilizers as well, and discard the one experimental outcome if we encounter syndromes. We would only see the codespace block of the reconstructed density matrix. We can effectuate such a density matrix by the setting `mode='code'`, but if we have obtained a density matrix with `mode='all'` or set `mode='custom'` and included the codespace in the keyword parameter `block_numbers`, we have all the information already. We could just extract the codespace block of such a state with the `.state` attribute, but its trace would not be 1, as the density matrix would typically be supported in other syndrome blocks.

Fortunately we can do any postselection after the fact. The function `postselect` takes a `pylove_state` object, and a new parameter `mode` and, for the case `mode=custom`, an optional keyword parameter `block_numbers`. The function then creates a new state with a density matrix according to the specified restrictions. In our case, we set `mode=code`. Let us take a look at the resulting fidelity.

In [ ]:
new_graphstate_post = postselect(state=new_graphstate, mode='code')
tr(new_graphstate.ideal(), new_graphstate_post)

The fidelity is not quite 1. The noise has corrupted the code space, but it has done so to a lesser degree than without postselection.
One last thing: every `pylove_state` instance has an attribute `n_entries`, that counts how many shots could actually be used in the construction of the density matrix. For states obtained with `mode=all`, this number equals the number of shots used for the simulation. However for hopelessly noisy states obtained with `mode=code`, one would be prudent to check if enough shots that were binned in the codespace to yield a representative result. Let's observe how the number of entries dropped from the $500$ shots we put into the simulation. 

In [ ]:
new_graphstate_post.n_entries

# Further customizing the simulation
### Custom state preparation

The `pylove_simulation` routine not just simulates the quantum circuit, but also the projective measurements. Since noise acts on all these circuits, one would like to reduce their depth, minimizing their exposition to error: the circuit depth of a `pylove_state` instance resulting a simulation can be retrieved with the attribute `.circuit_depth`. But how are the circuits compiled exactly? 

All subroutines, be it Pauli string rotations or measurements, follow the construction demonstrated on a Pauli string $X_0 Z_1 Z_2 Z_3 Z_4 Z_5 X_6$ below: an X-shaped construction of CNot gates with a $Z$-rotation or projective measurement in the center.

![here](subroutine.png)

The Pauli string rotation gadget $(2)$ are placed into the quantum circuit according to the order in which they are passed to `pylove_simulation`'s `quantum_circuit` argument. The same happens for the state preparation circuit. First all stabilizer measurements are placed into the circuit before the same happens to projective measurements associated with the logical operators. The orders of arguments `stabilizers` and `logical_operators` therefore obviously matters. However, there is more we can do.

Not only can we set arbitrary schedules for projective measurement gadgets of all state preparation operators (i.e. mix stabilizer generators with the logical operators), but define a different set of state preparation operators altogether; as long as the two sets of operators stabilize the same state. A reordered or entirely new set of state preparation operators can be passed to `pylove_simulation`'s keyword argument `state_prep_circuit`.

In our example, the stabilizer generators $(1)$ can be multiplied with the logical operators $(3)$ to eliminate all $Z$ operators.

In [ ]:
other_graphstate = pylove_simulation(
    stabilizers=stabs,
    logical_operators=log_ops,
    quantum_circuit=circuit,
    state_prep_circuit= log_ops + [     # here
        QubitOperator('X1', -1),        #
        QubitOperator('X3'),            #
        QubitOperator('X5', -1),        #
        QubitOperator('X7'),],          #
    shots=100,
    wire_noise=single_qubit_depolarizing(rate=.005),
    gate_noise=two_qubit_depolarizing(rate=.005),
    mode='all'
)
print('old circuit depth:', new_graphstate.circuit_depth)
print('new circuit depth:', other_graphstate.circuit_depth)

### Custom noise

We can define our own noise models. The simulator works with 1-qubit Pauli noise on quantum wires and 2-qubit Pauli noise on CNot gates.

Let us say we want to define 1-qubit noise channel: 
$$
{\rho} \; \mapsto \; 0.998 \,{\rho} + 0.001\,  X  {\rho} X  + 0.001\, Y {\rho} Y \, .
$$
This specific noise model can be passed to the simulator as lists containing its Pauli operators and coefficients. The Pauli operators are defined as `QubitOperator` instances with a single qubit. This includes the constant. Both lists are nested like demonstrated below.

In [ ]:
my_wire_noise = [
    [QubitOperator(()), QubitOperator('X0'), QubitOperator('Y0')],
    [.998, .001, .001]]

2-qubit noise channels such as
$$
{\rho} \; \mapsto \; 0.998 \,{\rho} + 0.001\,  Y_0 {\rho} Y_0  + 0.001\, Y_0 Y_1   {\rho}  Y_0 Y_1 \, .  
$$
are defined in the same way. This channel is applied after every CNot gate, where the qubit labelled `0` is the control qubit of the CNot, and the qubit labelled `1` is its target qubit. The gate noise model is defined below.

In [ ]:
my_gate_noise = [
    [QubitOperator(()), QubitOperator('Y0'), QubitOperator('Y0 Y1')],
    [.998, .001, .001]]

The two noise models can now be passed to the simulator.

In [ ]:
my_noisy_graphstate = pylove_simulation(
    stabilizers=stabs,
    logical_operators=log_ops,
    quantum_circuit=circuit,
    shots=100,
    wire_noise=my_wire_noise,       # here
    gate_noise=my_gate_noise,       # and here
    mode='all'
)